First we will get the dependencies and the Kohya_SS library

In [1]:
!git clone https://github.com/bmaltais/kohya_ss.git /kaggle/working/kohya_ss
%cd /kaggle/working/kohya_ss/
# We checkout this branch, because the current version (2023-09-04) requires too much memory
!git checkout 2853f4cec850557e83882a4072cfe416cb9c57d7
!/kaggle/working/kohya_ss/setup.sh -n
print("Kohya_SS is done setting up!")

Cloning into '/kaggle/working/kohya_ss'...
remote: Enumerating objects: 14874, done.
remote: Counting objects: 100% (506/506), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 14874 (delta 455), reused 393 (delta 390), pack-reused 14368 (from 4)
Receiving objects: 100% (14874/14874), 23.26 MiB | 31.33 MiB/s, done.
Resolving deltas: 100% (10443/10443), done.
/kaggle/working/kohya_ss
Note: switching to '2853f4cec850557e83882a4072cfe416cb9c57d7'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now a

# Set up training environment
1. Create a dataset called "training-images", put your images in there and save it.

In [6]:
# what you will use to trigger your lora
prompt_trigger = "wny_style"

import os
import shutil

# Create the directories
os.makedirs("/kaggle/working/training-data/img", exist_ok=True)
os.makedirs("/kaggle/working/training-data/reg", exist_ok=True)
os.makedirs("/kaggle/working/my-dreambooth-lora", exist_ok=True)
os.makedirs("/kaggle/working/training-data/log", exist_ok=True)

!rm -r /kaggle/working/training-data/*
# Copy the files
shutil.copytree("/kaggle/input/d/kartiksindura/training-images", f"/kaggle/working/training-data/img/25_{prompt_trigger}")

'/kaggle/working/training-data/img/25_wny_style'

# Time to train the model!
Just play the below code cell, it will require several hours to run the training.

In [10]:
%cd /kaggle/working/kohya_ss/
!accelerate launch --num_cpu_threads_per_process=2 "./sdxl_train_network.py" \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="/kaggle/working/training-data/img" \
  --output_dir="/kaggle/working/my-dreambooth-lora" \
  --logging_dir="/kaggle/working/training-data/log" \
  --resolution="1024,1024" \
  --network_alpha="16" \
  --network_dim=32 \
  --save_model_as=safetensors \
  --network_module=networks.lora \
  --text_encoder_lr=0.0004 \
  --unet_lr=0.0004 \
  --output_name="sdxl-dreambooth-lora" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --no_half_vae \
  --learning_rate="0.0004" \
  --train_batch_size="1" \
  --max_train_epochs=1 \
  --save_every_n_epochs="1" \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --optimizer_type="Adafactor" \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --max_data_loader_n_workers="0" \
  --gradient_checkpointing \
  --xformers \
  --bucket_no_upscale \
  --noise_offset=0.1 \
  --cache_latents \
  --cache_latents_to_disk \
  --lowram \
  --mem_eff_attn \
  --min_snr_gamma=5

/kaggle/working/kohya_ss
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
prepare tokenizers
prepare tokenizers
Using DreamBooth method.
Using DreamBooth method.
prepare images.
found directory /kaggle/working/training-data/img/25_wny_style contains 127 image files
No caption file found for 127 ima

# We now have the model weights (safetensors) in the `my-dreambooth-lora` folder to the right.